## Train Pointnet



## Install Dependencies

In [4]:
#It should be possible to run the notebook independent of anything else. 
# If dependency cannot be installed via pip, either:
# - download & install it via %%bash
# - atleast mention those dependecies in this section

import sys
!{sys.executable} -m pip install -q -e ../../utils/


You should consider upgrading via the '/group/cake/leo/.venv/bin/python -m pip install --upgrade pip' command.


In [5]:
!{sys.executable} -m pip install tqdm


You should consider upgrading via the '/group/cake/leo/.venv/bin/python -m pip install --upgrade pip' command.


### Import Dependencies

# System libraries

In [3]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
import sys
sys.setrecursionlimit(10000)

# Intialize tqdm to always use the notebook progress bar
import tqdm
tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml
import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import collections

import git


# Project utils

import aneurysm_utils
from aneurysm_utils import evaluation, training
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)
%config InlineBackend.figure_format='retina'  # adapt plots for retina displays

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
if "workspace" in os.getcwd():
    ROOT = "/workspace"
elif "/group/cake" in os.getcwd(): 
    ROOT = "/group/cake"


### Initialize Environment

In [5]:
env = aneurysm_utils.Environment(project="our-git-project", root_folder=ROOT)
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 
env.print_info()

Environment Info:

Library Version: 0.1.0
Configured Project: our-git-project

Folder Structure: 
- Root folder: /group/cake
 - Project folder: /group/cake/our-git-project
 - Datasets folder: /data/training
 - Models folder: /group/cake/our-git-project/models
 - Experiments folder: /group/cake/our-git-project/experiments


In [6]:
#define dataset and preprocessing parameters
dataset_params = {
    "prediction": "mask",
    "mri_data_selection": "", 
    "balance_data": False,
    "seed": 1,
    "resample_voxel_dim": (2.0,2.0,2.0),
    
}

preprocessing_params = {
    'min_max_normalize': True,
    'mean_std_normalize': False,
    'smooth_img': False, # can contain a number: smoothing factor
    'intensity_segmentation': 0.2
}

from aneurysm_utils.data_collection import load_aneurysm_dataset

df = load_aneurysm_dataset(
    env,
    mri_data_selection=dataset_params["mri_data_selection"],
    random_state=dataset_params["seed"]
)
df.head()

# Load MRI images and split into train, test, and validation
from aneurysm_utils.data_collection import split_mri_images


train_data, test_data, val_data, _ = split_mri_images(
    env, 
    df, 
    prediction=dataset_params["prediction"], 
    encode_labels=False,
    random_state=dataset_params["seed"],
    balance_data=dataset_params["balance_data"],
    resample_voxel_dim=dataset_params["resample_voxel_dim"],
    #resample_size=dataset_params["resample_size"]
)

mri_imgs_train, labels_train,_  = train_data
mri_imgs_test, labels_test,_  = test_data
mri_imgs_val, labels_val,_  = val_data



109
98
         Images
-----  --------
All         109
Train        87
Val          11
Test         11



  0%|          | 0/87 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [7]:
#get most common shape
from aneurysm_utils import preprocessing

most_common_shape=preprocessing.check_mri_shapes(mri_imgs_train)



Most common:
(70, 70, 60):      80
(35, 35, 31):       2
(36, 36, 30):       2
(55, 55, 49):       2
(36, 36, 31):       1


## Transform & Preprocess Data

In [8]:
from aneurysm_utils import preprocessing

size_of_train = len(mri_imgs_train)
size_of_test = len(mri_imgs_test)
size_of_val = len(mri_imgs_val)

# preprocess all lists as one to have a working mean_std_normalization
mri_imgs = mri_imgs_train + mri_imgs_test + mri_imgs_val
mri_imgs = preprocessing.preprocess(env, mri_imgs, preprocessing_params)

mri_imgs_train = mri_imgs[:size_of_train]
mri_imgs_train = [train for train in mri_imgs_train]
mri_imgs_test = mri_imgs[size_of_train : size_of_train + size_of_test]
mri_imgs_test = [test for test in mri_imgs_test]
mri_imgs_val = mri_imgs[size_of_train + size_of_test :]
mri_imgs_val = [val for val in mri_imgs_val]

# preprocess mask
x, y, h = labels_train[0].shape
labels_train = [label_train for label_train in labels_train]
labels_test = [label_test for label_test in labels_test]
labels_val = [label_val for label_val in labels_val]
# flatten

[INFO] Preprocessing: Min Max Normalize...
[INFO] Preprocessing: Intensity Segmentation...


In [9]:
size = most_common_shape#(93, 93, 80)  #(139, 139, 120)#(47,47,41)#
print(size)
train_index = [i for i, e in enumerate(mri_imgs_train) if e.shape != size]
mri_imgs_train = [i for j, i in enumerate(mri_imgs_train) if j not in train_index]
labels_train = [i for j, i in enumerate(labels_train) if j not in train_index]

test_index = [i for i, e in enumerate(mri_imgs_test) if e.shape != size]
mri_imgs_test = [i for j, i in enumerate(mri_imgs_test) if j not in test_index]
labels_test = [i for j, i in enumerate(labels_test) if j not in test_index]

val_index = [i for i, e in enumerate(mri_imgs_val) if e.shape != size]
mri_imgs_val = [i for j, i in enumerate(mri_imgs_val) if j not in val_index]
labels_val = [i for j, i in enumerate(labels_val) if j not in val_index]

mri_imgs_train[0].shape
preprocessing.check_mri_shapes(mri_imgs_train)



(70, 70, 60)
Most common:
(70, 70, 60):      80


(70, 70, 60)

## Train Model
Implementation, configuration, and evaluation of the experiment.

### Train Deep Model 3D data

In [10]:
from comet_ml import Optimizer

artifacts = {
    "train_data": (mri_imgs_train, labels_train),
    "val_data": (mri_imgs_val, labels_val),
    "test_data": (mri_imgs_test, labels_test)
}

params = {
    "batch_size": 32,
    "epochs": 1000,
    "learning_rate": 2.6e-5, # 3e-04, 1.0E-5
    "es_patience": None, # None = deactivate early stopping
    "weight_decay": 0.000003, # 1e-3
    "model_name": 'SegNet',
    "optimizer_momentum": 0.9,
    "optimizer":'Adam',
    "criterion": "CrossEntropyLoss", 
    "criterion_weights": [1.0, 30.0], # [1.75, 1.0],
    "sampler": None,   #'ImbalancedDatasetSampler2',
    "shuffle_train_set": True,
    "save_models":True,
    "scheduler": "ReduceLROnPlateau", # "ReduceLROnP
    "debug":False,
    "dropout":0.38,
    "start_radius":7.254,
    "sample_rate1": 0.2263,
    "sample_rate2": 0.2941,
    
    "save_models":True,
    "process": True,
    
}
params.update(dataset_params)
params.update(preprocessing_params)
#in these config dictionary you can uncomment and change the parameters that should be optimized
config = {
    # We pick the Bayes algorithm:
    "algorithm": "bayes",
    # Declare your hyperparameters in the Vizier-inspired format:
    "parameters": {
        #"criterion_weights": {"type": "integer", "scalingType": "loguniform", "min": 1, "max": 10},
        #"weight_decay": {"type": "float", "scalingType": "loguniform", "min": 1e-10, "max": 1e-3},
        "learning_rate": {"type": "float", "scalingType": "loguniform", "min": 1e-4, "max": 1e-2},
        #"scheduler": {"type": "categorical", "values": ["ReduceLROnPlateau", ""],},
        #"start_radius":{"type":"float","scalingType":"loguniform","min":0.1, "max":0.2},
    },
    # Declare what we will be optimizing, and how:
    "spec": {"metric": "bal_acc", "objective": "maximize"},  #test balance accuracy
}


opt = Optimizer(config, api_key=env.cached_data["comet_key"])

COMET INFO: COMET_OPTIMIZER_ID=84815cc0af994054b5452060fe57ae28
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': '84815cc0af994054b5452060fe57ae28', 'lastUpdateTime': None, 'maxCombo': 0, 'name': '84815cc0af994054b5452060fe57ae28', 'parameters': {'learning_rate': {'max': 0.01, 'min': 0.0001, 'scalingType': 'loguniform', 'type': 'float'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 0, 'metric': 'bal_acc', 'minSampleSize': 100, 'objective': 'maximize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 15621022982, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.1'}


In [11]:
#collect garbage
import gc
gc.collect()

294

In [ ]:
# Finally, get experiments, and train your models, also uncomment or add coresponding entries to param_copy
# that should be optimzied
import time
for comet_exp in opt.get_experiments(project_name=env.project):
    print(comet_exp)
    param_copy = params.copy()
    comet_exp.params
    
    #param_copy["weight_decay"] = comet_exp.get_parameter("weight_decay")
    #param_copy["criterion_weights"] = comet_exp.get_parameter("criterion_weights")
    param_copy["learning_rate"] = comet_exp.get_parameter("learning_rate")
    #param_copy["scheduler"] = comet_exp.get_parameter("scheduler")

    exp = env.create_experiment(
        params["prediction"] + "-pytorch-" + params["model_name"], comet_exp
    ) #params["selected_label"] + "-hyperopt-" + params["model_name"]
    exp.run(training.train_pytorch_model, param_copy, artifacts)
    
    time.sleep(3)
    del exp
    import gc
    gc.collect()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/rbendias/our-git-project/301d3f49c82c4e72bf9413d27858d230



[INFO] Experiment mask-pytorch-SegNet is initialized.
[INFO] Running experiment: 2021-07-21-16-34-27_mask-pytorch-segnet
Number of Classes 2
Selected model: SegNet
